## 结合工具规划逻辑

包括 ReAct、ReWoo、Plan 等复杂的工具规划推理也可以通过 FlowAgent 实现。

## ReAct

In [7]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReAct(planner=ChatQwen(), tools=[get_city, get_weather, booking])
flow("我在鸿蒙公司，帮我查询一下天气情况")
flow.agents[0].memory

[AGENT] STEP 1 >>> Node 1: ChatQwen.4723354368
**思考** 
由于需要查询的是鸿蒙公司所在地的天气情况，首先我需要确定鸿蒙公司的具体位置是在哪个城市。这一步需要用到 `get_city` 函数来通过提供的地址信息（这里是鸿蒙公司）获取所在城市的信息。

**规划** 
为了找出鸿蒙公司所在的城市，我的第一步应该是调用 `get_city` 函数，并提供鸿蒙公司作为位置参数。

**行动** 
Plan: 获取鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]
tool_to_exec {'function': {'name': 'get_city', 'arguments': '{"location": "鸿蒙公司"}'}}
广州


**观察** 上面的行动结果为:
广州
[AGENT] STEP 2 >>> Node 1: ChatQwen.4723354368
**思考** 
既然已经知道鸿蒙公司位于广州，那么接下来的任务就是查询广州的天气情况了。这一步骤需要用到 `get_weather` 函数，并且要向它提供广州作为城市参数。

**规划** 
下一步的计划是查询广州的天气情况，为此需要调用 `get_weather` 函数，并设置参数 `city` 为广州。

**行动** 
Plan: 查询广州的天气情况. #E2 = get_weather[{"city": "广州"}]
tool_to_exec {'function': {'name': 'get_weather', 'arguments': '{"city": "广州"}'}}
广州天气不错，一直是大晴天


**观察** 上面的行动结果为:
广州天气不错，一直是大晴天
[AGENT] STEP 3 >>> Node 1: ChatQwen.4723354368
**思考** 
既然已经得到了广州的天气情况，即“广州天气不错，一直是大晴天”，这就意味着我已经完成了查询鸿蒙公司所在地天气的任务。

**规划** 
现在可以输出最终的答案。

**行动** 
整理并输出最终答案。

<final_answer>
鸿蒙公司所在的广州天气不错，一直是大晴天。
</final_answer>
最终答案为: ['鸿蒙公

[{'role': 'system',
  'content': '尽你所能完成任务。\n\n**你要解决的问题是: ** 我在鸿蒙公司，帮我查询一下天气情况\n\n\n\n**思考** \n由于需要查询的是鸿蒙公司所在地的天气情况，首先我需要确定鸿蒙公司的具体位置是在哪个城市。这一步需要用到 `get_city` 函数来通过提供的地址信息（这里是鸿蒙公司）获取所在城市的信息。\n\n**规划** \n为了找出鸿蒙公司所在的城市，我的第一步应该是调用 `get_city` 函数，并提供鸿蒙公司作为位置参数。\n\n**行动** \nPlan: 获取鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]\n\n**观察** 上面的行动结果为:\n广州\n\n**思考** \n既然已经知道鸿蒙公司位于广州，那么接下来的任务就是查询广州的天气情况了。这一步骤需要用到 `get_weather` 函数，并且要向它提供广州作为城市参数。\n\n**规划** \n下一步的计划是查询广州的天气情况，为此需要调用 `get_weather` 函数，并设置参数 `city` 为广州。\n\n**行动** \nPlan: 查询广州的天气情况. #E2 = get_weather[{"city": "广州"}]\n\n**观察** 上面的行动结果为:\n广州天气不错，一直是大晴天\n\n\n(现在不要急于解决问题，而是继续按照如下步骤一步一步输出你的推理过程。)\n\n**思考** \n对当前情况进行反思, 然后说明你现在的决策：当前就结束任务并输出最终答案，还是继续下一个子任务。\n\n请你按如下格式整理最终答案:\n<final_answer>\n(在此输出你的最终答案)\n</final_answer>\n\n如果已经输出最终答案，就必须立即停止输出。\n\n**规划** \n你可以尝试直接解决下一步的问题；\n但如果涉及工具擅长擅长的领域，即使任务简单也必须使用工具回调。\n\n如果你尝试自己解决问题，请将详细过程输出在如下位置：\n\n(在此输出你自己解决的子任务目标和结果)\n\n**行动** \n你可以从 ** get_city,get_weather,booking ** 中选择一个工具来建立一个行动清单。\n\n这些工具的详细描述如

In [2]:
print(flow.completed_work)


<final_answer>
已经为你预订好了广州的羽毛球场，祝你玩得愉快！
</final_answer>最终答案为: ['已经为你预订好了广州的羽毛球场，祝你玩得愉快！']


In [7]:
flow.max_steps

20

## ReWOO

- 管道计算场景：上一个任务的输出，作为下一个任务的输入
- 参数组装场景：上一个任务的输出需要做一定转换，才能作为下一个任务的输入（执行哪个任务是确定的，但参数需要组装）
- 条件分支场景：上一个任务的输出作为一种判定条件，决定执行接下来哪个任务（执行哪个任务不确定，需要根据当前任务返回来判定）

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReWOO
def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReWOO(planner=ChatQwen(), tools=[get_city, get_weather, booking])
flow("我在鸿蒙公司，帮我查询一下天气情况")
flow.agents[0].memory

[AGENT] STEP 1 >>> Node 1: planner
Plan: 首先，我们需要确定鸿蒙公司的位置所在城市。
#E1 = get_city({"location": "鸿蒙公司"})

Plan: 使用上一步获取的城市信息，查询该城市的天气情况。
#E2 = get_weather({"city": "#E1"})
tool_to_exec {'function': {'name': 'get_city', 'arguments': '{"location": "鸿蒙公司"}'}}
广州


**观察** 上面的行动结果为:
广州
tool_to_exec {'function': {'name': 'get_weather', 'arguments': '{"city": "广州"}'}}
广州天气不错，一直是大晴天


**观察** 上面的行动结果为:
广州天气不错，一直是大晴天


[{'role': 'system',
  'content': 'For the following task, make plans that can solve the problem step by step. \nFor each plan, indicate which external tool together with tool input to retrieve evidence. \nYou can store the evidence into a variable #E that can be called by later tools. \n(Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}\n{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}\n{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["re